# Задача о знакомствах. Ищем через сколько ссылок в википедии можно выйти на нужного человека. (в данном примере для Пугачевой и Трампа

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import numpy as np

Пишем функцию getLinks, которая возвращает список ссылок лежащщих на странице в википедии

In [2]:
def getLinks(articleUrl):
    list_uf_urls = []
    html = urlopen('http://en.wikipedia.org{}'.format(articleUrl))
    bs = BeautifulSoup(html, 'html.parser')
    for link in bs.find_all('a', href=re.compile('^(/wiki/)((?!:).)*$')):
        if 'href' in link.attrs:
            list_uf_urls.append(link.attrs['href'])
    return list_uf_urls

In [4]:
### введите ссылку с которой хотите начать поиск

url = '/wiki/Alla_Pugacheva'

###
html = urlopen('http://en.wikipedia.org{}'.format(url))
bS = BeautifulSoup(html, 'html.parser')
count = 0
while (bS.find_all(text='Trump') == [] and count < 1000):
    links = getLinks(url)
    url = np.random.choice(links)
    html = urlopen('http://en.wikipedia.org{}'.format(url))
    bS = BeautifulSoup(html, 'html.parser')
    count = count + 1

In [5]:
count

247

In [6]:
url

'/wiki/John_F._Kennedy'

# Функция, ищущая все внешние и веутренние ссылки на странице  

In [2]:
def is_valid(url):
    """
    Проверяет, является ли url допустимым
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [3]:
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import colorama

# запускаем модуль colorama
colorama.init()

GREEN = colorama.Fore.GREEN
GRAY = colorama.Fore.LIGHTBLACK_EX # цвета 
RESET = colorama.Fore.RESET
RED = colorama.Fore.RED

internal_urls = set()
external_urls = set()

total_urls_visited = 0


def is_valid(url):
    """
    Проверка url
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)


def get_all_website_links(url):
    """
    Возвращает все найденные URL-адреса на `url, того же веб-сайта.
    """
    # все URL-адреса `url`
    urls = set()
    # доменное имя URL без протокола
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            # пустой тег href
            continue
        # присоединяемся к URL, если он относительный (не абсолютная ссылка)
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        # удалить параметры URL GET, фрагменты URL и т. д.
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_valid(href):
            # неверный URL
            continue
        if href in internal_urls:
            # уже в наборе
            continue
        if domain_name not in href:
            # внешняя ссылка
            if href not in external_urls:
                print(f"{RED}[!] Внешняя ссылка: {href}{RESET}")
                external_urls.add(href)
            continue
        print(f"{GREEN}[*] Внутреннея ссылка: {href}{RESET}")
        urls.add(href)
        internal_urls.add(href)
    return internal_urls, external_urls

URLPARSE - именованный кортеж, делящий ссылку на ...

In [4]:
urlparse('https://sites.google.com/view/qm1-hse/')

ParseResult(scheme='https', netloc='sites.google.com', path='/view/qm1-hse/', params='', query='', fragment='')

# Сохраняем фото Дональда Трампа (парсинг медиа с интернета)

In [ ]:
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup
photos = []
html = urlopen('https://en.wikipedia.org/wiki/Donald_Trump')
bs = BeautifulSoup(html, 'html.parser')
imageLocation = bs.find_all('img')
for ek in imageLocation:
    photos.append(ek['src'])

urlretrieve('https:'+ photos[1], 'logo.jpg')

# Скачиваем таблицу из википедии (парсинг таблиц)

In [ ]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('https://en.wikipedia.org/wiki/Euler%27s_totient_function')
bs = BeautifulSoup(html, 'html.parser')
table = bs.find_all('table', {'class': 'wikitable'})[0]
rows = table.find_all('tr')
csvFile = open('editors.csv', 'wt+')
writer = csv.writer(csvFile)
csvRow = []
try:
    for row in rows:
        for cell in row.find_all(['td', 'th']):
            csvRow.append(cell.get_text())
            writer.writerow(csvRow)
finally:
    width = len(row.find_all(['td', 'th']))
    csvFile.close()

In [ ]:
print(width)

In [ ]:
with open ('file.txt', 'w') as f:
    for i in range (1, len(csvRow)):
        if i % width == 0:
            f.write(csvRow[i])
            f.write('\n')
   
        elif(i % width == (width-1) ):
            f.write(csvRow[i].replace('\n', ''))
        else:
            f.write(csvRow[i].strip() + '±')
            
f.close()    

In [ ]:
import pandas as pd
data = pd.read_csv('file.txt', sep='±')


In [ ]:
data

# Данная программа ищет текст (слово) на всех ссылках с данного сайта и возвращет ссылки, где текст был найден

In [5]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse, urljoin


def is_valid(url):
    """
    Проверяет, является ли url допустимым
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)


class Website:

    def __init__(self, name, url, tag, bodyTag):
        self.name = name
        self.url = url
        self.tag = tag
        self.bodyTag = bodyTag


class Crawler(Website):
    internal_urls = set()
    external_urls = set()
    objects = set()

    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BeautifulSoup(req.text, 'html.parser')

    def get_all_website_links(self, url, internal_urls=internal_urls):
        urls = set()
        domain_name = urlparse(url).netloc
        soup = self.getPage(url)
        for a_tag in soup.findAll("a"):
            href = a_tag.attrs.get("href")
            if href == "" or href is None:
                continue
            href = urljoin(url, href)
            parsed_href = urlparse(href)
            href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
            if not is_valid(href):
                continue
            if href in internal_urls:
                continue
            urls.add(href)
            internal_urls.add(href)
        internal_urls.add(url)
        return internal_urls

    def One_link(self, url, objects=objects):

        bs = self.getPage(url)
        if bs is not None:
            bs = bs.findAll(self.tag)
            for el in bs:
                if self.bodyTag in el.get_text():
                    objects.add(url)
        return objects

    def crawler(self, objects=objects):

        internal_links = self.get_all_website_links(self.url)
        for link in internal_links:
            self.One_link(link)
        return objects


site = Crawler('Times', 'https://www.littre.org', 'p', 'peut')
for el in site.crawler():
    print(el)


https://www.littre.org/definition/dire
https://www.littre.org/definition/race
https://www.littre.org/annexes/preface
https://www.littre.org
https://www.littre.org/annexes/causerie
http://creativecommons.org/licenses/by-sa/3.0/deed.fr
https://www.littre.org/
https://www.littre.org/definition/garce
https://www.littre.org/etymologie
https://www.littre.org/definition/%C3%A9viter
https://www.littre.org/definition/libert%C3%A9
